# Prepping data

In [9]:
# file imports
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.optimize import minimize as minimize
from itertools import islice

# splitting data in train test and val set
def data_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)  
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
    return X_train, X_test, X_val, y_train, y_test, y_val

data = pd.read_excel(r'C:\Users\RE-Giorgio\Downloads\dataPoints.xlsx')
X = np.array(data.iloc[:,:2])
y = np.array(data.iloc[:, 2])
X_train, X_test, X_val, y_train, y_test, y_val = data_split(X, y)

# Neural Network parent class

In [13]:
class NeuralNetwork:
    
    def __init__(self, X, y, N, sigma):

        self.X = X
        self.y = y
        self.N = N
        self.bias = np.zeros(self.N)
        self.weights1 = np.random.normal(0,1,(self.X.shape[1], self.N))
        self.weights2 = np.random.normal(0,1,(self.N, 1))
        self.output = np.zeros(y.shape[0])
        self.rho = np.random.normal(10e-4,10e-5,1)
        self.sigma = sigma
    
    # needed to apply the norm on the parameters' vectors all togheter for the regularization
    def concatenate(self,l):
        l = [np.array(array).reshape(-1,1) for array in l]
        return np.concatenate(l)
    

    def loss(self,params, fixed_params):
        
        # the next 2 lines are two divide the concatenated vectors for the forward propagation
        seclist = [self.X.shape[1]*self.N, self.N, self.N]
        weights1, weights2, bias = [list(islice(iter(params), 0, i)) for i in seclist]
        y, rho, sigma = fixed_params
        return 0.5 * np.mean(np.square((self.act_fun(self.forward_propagation()) - y))) +\
            rho*np.square(np.linalg.norm(self.concatenate([weights1, weights2, bias])))

## MLP child class

In [14]:
class Mlp(NeuralNetwork):
    
    def act_fun(self,x):
        return (1-np.exp(-2*x*self.sigma))/(1+np.exp(-2*x*self.sigma))

    def forward_propagation(self):
        
        self.hidden_layer = self.act_fun(np.dot(self.X, self.weights1))
        self.output = np.dot(self.hidden_layer, self.weights2)
        return self.output
                          
    def optimization(self):
        function_args = np.array([self.y, self.rho, self.sigma])
        inits = self.concatenate([self.weights1, self.weights2, self.bias])
        return minimize(self.loss, x0 = inits, method='CG', args = function_args)
        

## first run

In [15]:
nn = Mlp(X_train, y_train, 10, 1)
nn.optimization()

     fun: 1.674591715823805
     jac: array([-1.68383121e-06,  7.00354576e-07,  1.19209290e-07, -2.53319740e-07,
        1.11758709e-06, -4.17232513e-07,  2.57790089e-06,  5.06639481e-07,
        9.38773155e-07, -9.83476639e-07, -1.34110451e-07, -5.81145287e-07,
        4.61935997e-07, -5.96046448e-08,  4.91738319e-07,  2.23517418e-07,
        2.23517418e-07,  3.87430191e-07,  2.53319740e-07,  5.96046448e-08,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 1092
     nit: 5
    njev: 26
  status: 0
 success: True
       x: array([-2.55695756e-04,  1.08601523e-04,  1.86828097e-05, -3.86390501e-05,
        1